# Филтрирање на слика во фреквенциски домен

## Теорија

За да филтрираме слика во фреквенциски домен потребно е да ја помножиме сликата во фреквенциски домен со филтерот (кернел) кој е исто така во фреквенциски домен.

$$I_{f,filt} = HI_f$$

За таа цел потребно е да се вметне Octave функција која ќе креира “Band pass gaussian filter” во фреквенциски домен. Bandpass гаусовиот филтер во фреквенциски домен е дефиниран со равенката:

$$ H(i,j) = e^{-\frac{d(i,j)^2}{f_h^2}} (1 - e^{-\frac{d(i,j)^2}{f_l^2}}) $$

Каде што 𝑑(𝑖,𝑗) е евклидовото растојанието на даден пиксел од центарот на сликата, додека 𝑓ℎ и 𝑓𝑙 се праговите на високите и ниските фреквенции кои сакаме да ги филтрираме. 
Конверзија на 2D сликата од просторен во фреквенциски домен се прави со користење на Фуриева трансформација. Фуриевата трансформација е дефинирана како:

$$ F(u,v) = \iint_{-\infty}^{\infty} f(x,y) e^{-i2\pi(ux + vy)} \,dx\,dy$$

А во дискретниот домен горната равенка може да биде преведена како

$$ F(u,v) = \sum_{m=-\infty}^{\infty} \sum_{n=-\infty}^{\infty} f[m,n] \cdot e^{-i2\pi(umx_0 + vny_0)} $$

Конверзија на 2D сликата од фреквенциски во просторен домен се прави со користење на инверзна Фуриева трансформација. Инверзната Фуриева трансформација е дефинирана како:

$$ f(x,y) = \iint_{-\infty}^{\infty} F(u,v) e^{i2\pi(ux + vy)} \,du\,dv$$

А во дискретниот домен горната равенка може да биде преведена како

$$ f(x,y) = \sum_{m=-\infty}^{\infty} \sum_{n=-\infty}^{\infty} F(m,n) \cdot e^{i2\pi(xmu_0 + ynv_0)} $$

## Имплементација

Функција за создавање на филтер:

In [1]:
function filter = create_filter(nx,ny,d0,d1)
    filter = ones(nx,ny);

    for i = 0:nx-1
        for j = 0:ny-1
            dist= sqrt((i-nx/2)^2 + (j-ny/2)^2);
            filter(i+1,j+1) = exp(-(dist^2)/(d1^2)).*(1.0-exp(-(dist^2)/(d0^2)));
        end
    end

    filter(nx/2+1,ny/2+1)=1;
end

Вчитување на сликата и Фуриева трансформација врз неа:

In [2]:
global img img_fft
img = imread('Barbara.tif');
img_fft = fftshift(fft2(img));

Функција за добивање на филтрирана слика (параметри се фрекфенциските прагови со чекор 10):

In [3]:
function filtered_image = filter_image(filter)
    global img_fft
    filtered_image = abs(ifft2(ifftshift(filter.*img_fft)));
end

Функција за добивање на филтерот во фрекфенциски домен (параметри се фрекфенциските прагови со чекор 10):

In [4]:
function filter = filter_freq(l, h)
    global img
    
    fl = 1 + l*10;
    fh = 1 + h*10;
    
    img_double = im2double(img);
    [nx, ny] = size(img_double);
    
    filter = create_filter(nx, ny, fl, fh);
end

Креирање на паровите слика и филтер кои ке се користат во визуелизација, границите на слајдерот се од 0 до 10, а бидејќи високата граница не може да биде помала од ниската, вториот for циклус е поставен на дадениот начин:

In [ ]:
for i = 0:10
    for j = i:10
         tmp_f = filter_freq(i, j);
         tmp_i = filter_image(tmp_f);
         images(i+1, j+1, :) = tmp_i(:);
         filters(i+1, j+1, :) = tmp_f(:);
    end
end

In [ ]:
save('f_images.mat', 'images', '-v6')
save('f_filters.mat', 'filters', '-v6')

## Визуелизација

In [ ]:
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import numpy as np
from ipywidgets import interactive, widgets, interact, HBox, Label
from IPython.display import Javascript

images = sio.loadmat('f_images')['images']
filters = sio.loadmat('f_filters')['filters']

slider = widgets.IntRangeSlider(
    min=0,
    max=10,
    step=1,
    readout=False,
    description=' ')
slider.layout.width = '500px'

def update_freq_cutoffs(y):
    L, H = y[0], y[1]
    fig = plt.figure()
    a = fig.add_subplot(1, 2, 1)
    imgplot = plt.imshow(np.rot90((images[L][H].reshape(402,566)), -1))
    a = fig.add_subplot(1, 2, 2)
    imgplot = plt.imshow(np.rot90((filters[L][H].reshape(402,566)), -1))

HBox([Label('Frequency cutoffs:'), interactive(update_freq_cutoffs, y=slider)])